In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_41.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 34.1 s, sys: 2.97 s, total: 37.1 s
Wall time: 7.79 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 182 ms, sys: 273 ms, total: 454 ms
Wall time: 453 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 684 µs, sys: 81 µs, total: 765 µs
Wall time: 779 µs


In [7]:
coef_lgb = 0.8
coef_mlp = 0.2

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 326 ms, sys: 570 ms, total: 896 ms
Wall time: 893 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000173,0.368493,0.000349,0.198581,0.108626,0.000398,0.123283,0.000255,0.001475,0.015717,0.002240,0.179606,0.000256,0.000548,0.245786
3492886,130787971,0.000187,0.013098,0.000165,0.053464,0.169525,0.000225,0.021695,0.367642,0.000362,0.036712,0.000766,0.329667,0.000257,0.006235,0.233158
3492887,130787974,0.000735,0.022421,0.000135,0.852516,0.031650,0.000326,0.078459,0.000045,0.000142,0.001312,0.000361,0.009741,0.000190,0.001966,0.099989
3492888,130788053,0.000059,0.724992,0.000074,0.149644,0.003393,0.000110,0.010439,0.080236,0.000433,0.000368,0.000237,0.029257,0.000102,0.000657,0.161684
3492889,130788054,0.000196,0.084732,0.000288,0.413864,0.189551,0.000477,0.057311,0.000105,0.000509,0.021892,0.001129,0.228524,0.000635,0.000787,0.236733


In [13]:
%%time
test_ensemble.to_csv('test_pred_42.csv',index=False)

CPU times: user 1min 26s, sys: 947 ms, total: 1min 27s
Wall time: 1min 27s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_42.csv -m "Message"

100%|██████████████████████████████████████| 1.06G/1.06G [04:55<00:00, 3.86MB/s]
Successfully submitted to PLAsTiCC Astronomical Classification